# Testing
* shorting strategy
* making it paper tradeable

In [1]:
from evolution_operators import *
from utility_functions import *

In [21]:
def evaluate_strategy(data, fast_period, slow_period, fast_p_exit, slow_p_exit,rsi_period,rsi_derivative_threshold, stop_loss_percentage):
    # Calculate indicators
    print(fast_period,slow_period,fast_p_exit, slow_p_exit, stop_loss_percentage)
    data['MACD'], data['Signal'] = calculate_macd(data, fast_period, slow_period)
    data['MACD_E'], data['Signal_E'] = calculate_macd(data, fast_p_exit, slow_p_exit)

    data['ROC'] = calculate_roc(data, 3)
    data['RSI_Derivative'] = calculate_rsi_derivative(data, rsi_period)
    
    # Apply trailing stop-loss exit condition
#     stop_loss = trailing_stop_loss(data, stop_loss_percentage)
    
    # Simulated trading logic
    position = 0  # 0 for no position, 1 for long position
    profit = 0
    total_trades = 0
    winning_trades = 0
    
    stop_loss_price = 0
    
    for i in range(1, len(data)):
        try:
            if data['MACD'][i] < data['Signal'][i] and data['RSI_Derivative'][i] > rsi_derivative_threshold:#and data['ROC'][i] > 1: #and 
                if position == 0:
                    position = 1  # Buy (enter long position)
                    entry_price = data['Close'][i]
                    stop_loss_price = entry_price + (entry_price * 0.01 * stop_loss_percentage)
                    print(f"Entry: {entry_price} on {data.index[i]} Stop Loss: {stop_loss_price}")
                    total_trades += 1  
            elif position == 1:
                if data['Close'][i] < entry_price:  # Adjust stop loss only when the price goes up
                    stop_loss_price = max(stop_loss_price, data['Close'][i] + (data['Close'][i] * 0.01 * stop_loss_percentage))

                if data['High'][i] > stop_loss_price:
                    position = 0  # Sell due to stop loss
                    exit_price = data['Open'][i] if stop_loss_price < data['Open'][i] else stop_loss_price
                    profit += -1*(exit_price - entry_price)  # Calculate profit/loss)
                    print(f"Exit: {exit_price} on {data.index[i]} exited on stop loss")
                    print(f"Profit: {profit}")
                    if(exit_price<= entry_price):
                        winning_trades += 1
                elif data['MACD_E'][i] > data['Signal_E'][i]:
                    position = 0  # Sell due to exit condition
                    exit_price = data['Close'][i]
                    profit += -1*(exit_price - entry_price)  # Calculate profit/loss
                    print(f"Exit: {exit_price} on {data.index[i]} exited on MACD condition")
                    print(f"Profit: {profit}")
                    if(exit_price<= entry_price):
                        winning_trades += 1
        except Exception as ex:
            print("Something caused the error",ex)
            break
        win_rate = winning_trades/total_trades if total_trades != 0 else 0
    
    return [profit,total_trades,winning_trades,win_rate]

In [23]:
# Fetch historical stock data
ticker_symbol = '^NSEI'  # Replace with your desired stock symbol
start_date = '2022-01-01'
end_date = '2022-12-31'
stock_data = fetch_stock_data(ticker_symbol, start_date, end_date)

# Genetic Algorithm parameters
population_size = 200
num_generations = 20
mutation_rate = 0.1
elite_size = 50
consecutive_threshold = 3  # Number of consecutive generations with no fitness improvement to consider saturation
fitness_history = []  # To store fitness scores across generations

# GA: Initialization
def generate_individual():
    fast_p = random.randint(2,10)
    slow_p = random.randint(fast_p,20)
    fast_p_exit = random.randint(2,10)
    slow_p_exit = random.randint(fast_p_exit,20)
    return [fast_p,slow_p,fast_p_exit,slow_p_exit, random.randint(2,30),random.randint(-50,50),random.randint(0,3)]


# Known or useful parameter values
known_parameters = [
#     [3, 4, 1, 10,0],
#     [7, 9, 5,9,  0],
#     [7, 9, 5,9,  3],
#     [2, 3, 2,4,  0],
#     [2, 15, 6, 10, 0],
    [5, 13, 8, 11, 14,-20,0]# Example set of known parameters
                     # Add more known parameter sets if available
]

# Randomly generate the rest of the population
remaining_population = [generate_individual() for _ in range(population_size - len(known_parameters))]

# Initialize population using known parameter values and randomly generated values
population = known_parameters + remaining_population
# population = [generate_individual() for _ in range(population_size)]


    
for generation in range(num_generations):
    fitness_scores = [evaluate_strategy(stock_data.copy(), *individual)[0] for individual in population]
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]

    new_population = []

    # Select the top K individuals (elite) for the next generation
    elite = sorted_population[:elite_size]

    # Check for convergence or other stopping criteria
    fitness_history.append(max(fitness_scores))

    # Check for convergence or other stopping criteria
    if len(fitness_history) >= consecutive_threshold:
        recent_improvements = fitness_history[-consecutive_threshold:]
        if len(set(recent_improvements)) == 1:
            print("Saturation condition met, breaking the loop")
            break

    # Create new population by generating offspring from the elite individuals
    new_population = generate_offspring(elite,population_size,mutation_rate)

    population = new_population
    

# Find the best individual (parameters) from the final population
best_individual_index = np.argmax(fitness_scores)
best_individual = population[best_individual_index]
best_fitness_score = fitness_scores[best_individual_index]

print("Best Parameters:", best_individual)
print("Best Fitness Score:", best_fitness_score)


[*********************100%%**********************]  1 of 1 completed
5 13 8 11 0
Entry: 17576.849609375 on 2022-02-01 00:00:00 Stop Loss: 17576.849609375
Exit: 17706.19921875 on 2022-02-02 00:00:00 exited on stop loss
Profit: -129.349609375
Entry: 17266.75 on 2022-02-08 00:00:00 Stop Loss: 17266.75
Exit: 17370.099609375 on 2022-02-09 00:00:00 exited on stop loss
Profit: -232.69921875
Entry: 17352.44921875 on 2022-02-15 00:00:00 Stop Loss: 17352.44921875
Exit: 17408.44921875 on 2022-02-16 00:00:00 exited on stop loss
Profit: -288.69921875
Entry: 16658.400390625 on 2022-02-25 00:00:00 Stop Loss: 16658.400390625
Exit: 16658.400390625 on 2022-03-02 00:00:00 exited on stop loss
Profit: -288.69921875
Entry: 16013.4501953125 on 2022-03-08 00:00:00 Stop Loss: 16013.4501953125
Exit: 16757.099609375 on 2022-03-10 00:00:00 exited on stop loss
Profit: -1032.3486328125
Entry: 17784.349609375 on 2022-04-08 00:00:00 Stop Loss: 17784.349609375
Exit: 16240.2998046875 on 2022-05-18 00:00:00 exited on MA

Entry: 17277.94921875 on 2022-01-25 00:00:00 Stop Loss: 17796.2876953125
Exit: 17780.0 on 2022-02-02 00:00:00 exited on MACD condition
Profit: -502.05078125
Entry: 17266.75 on 2022-02-08 00:00:00 Stop Loss: 17784.7525
Exit: 17463.80078125 on 2022-02-09 00:00:00 exited on MACD condition
Profit: -699.1015625
Entry: 17352.44921875 on 2022-02-15 00:00:00 Stop Loss: 17873.0226953125
Exit: 17304.599609375 on 2022-02-17 00:00:00 exited on MACD condition
Profit: -651.251953125
Entry: 16658.400390625 on 2022-02-25 00:00:00 Stop Loss: 17158.15240234375
Exit: 16594.900390625 on 2022-03-10 00:00:00 exited on MACD condition
Profit: -587.751953125
Entry: 17784.349609375 on 2022-04-08 00:00:00 Stop Loss: 18317.88009765625
Exit: 16240.2998046875 on 2022-05-18 00:00:00 exited on MACD condition
Profit: 956.2978515625
Entry: 16478.099609375 on 2022-06-09 00:00:00 Stop Loss: 16972.44259765625
Exit: 15699.25 on 2022-06-24 00:00:00 exited on MACD condition
Profit: 1735.1474609375
Entry: 16049.2001953125 on 

4 6 7 13 3
Entry: 17277.94921875 on 2022-01-25 00:00:00 Stop Loss: 17796.2876953125
Exit: 17560.19921875 on 2022-02-03 00:00:00 exited on MACD condition
Profit: -282.25
Entry: 17266.75 on 2022-02-08 00:00:00 Stop Loss: 17784.7525
Exit: 17605.849609375 on 2022-02-10 00:00:00 exited on MACD condition
Profit: -621.349609375
Entry: 17352.44921875 on 2022-02-15 00:00:00 Stop Loss: 17873.0226953125
Exit: 17304.599609375 on 2022-02-17 00:00:00 exited on MACD condition
Profit: -573.5
Entry: 17063.25 on 2022-02-23 00:00:00 Stop Loss: 17575.1475
Exit: 16594.900390625 on 2022-03-10 00:00:00 exited on MACD condition
Profit: -105.150390625
Entry: 17222.0 on 2022-03-28 00:00:00 Stop Loss: 17738.66
Exit: 17498.25 on 2022-03-30 00:00:00 exited on MACD condition
Profit: -381.400390625
Entry: 17784.349609375 on 2022-04-08 00:00:00 Stop Loss: 18317.88009765625
Exit: 16240.2998046875 on 2022-05-18 00:00:00 exited on MACD condition
Profit: 1162.6494140625
Entry: 16569.55078125 on 2022-06-06 00:00:00 Stop L

Entry: 17339.849609375 on 2022-01-31 00:00:00 Stop Loss: 17860.04509765625
Exit: 17560.19921875 on 2022-02-03 00:00:00 exited on MACD condition
Profit: -220.349609375
Entry: 17266.75 on 2022-02-08 00:00:00 Stop Loss: 17784.7525
Exit: 17463.80078125 on 2022-02-09 00:00:00 exited on MACD condition
Profit: -417.400390625
Entry: 17352.44921875 on 2022-02-15 00:00:00 Stop Loss: 17873.0226953125
Exit: 17322.19921875 on 2022-02-16 00:00:00 exited on MACD condition
Profit: -387.150390625
Entry: 16658.400390625 on 2022-02-25 00:00:00 Stop Loss: 17158.15240234375
Exit: 16594.900390625 on 2022-03-10 00:00:00 exited on MACD condition
Profit: -323.650390625
Entry: 17784.349609375 on 2022-04-08 00:00:00 Stop Loss: 18317.88009765625
Exit: 16259.2998046875 on 2022-05-17 00:00:00 exited on MACD condition
Profit: 1201.3994140625
Entry: 16478.099609375 on 2022-06-09 00:00:00 Stop Loss: 16972.44259765625
Exit: 15556.650390625 on 2022-06-23 00:00:00 exited on MACD condition
Profit: 2122.8486328125
Entry: 1

9 19 7 13 3
Entry: 17352.44921875 on 2022-02-15 00:00:00 Stop Loss: 17873.0226953125
Exit: 17304.599609375 on 2022-02-17 00:00:00 exited on MACD condition
Profit: 47.849609375
Entry: 16658.400390625 on 2022-02-25 00:00:00 Stop Loss: 17158.15240234375
Exit: 16594.900390625 on 2022-03-10 00:00:00 exited on MACD condition
Profit: 111.349609375
Entry: 17136.55078125 on 2022-04-20 00:00:00 Stop Loss: 17650.6473046875
Exit: 15809.400390625 on 2022-05-19 00:00:00 exited on MACD condition
Profit: 1438.5
Entry: 15350.150390625 on 2022-06-20 00:00:00 Stop Loss: 15810.65490234375
Exit: 15926.2001953125 on 2022-06-27 00:00:00 exited on stop loss
Profit: 862.4501953125
Entry: 17577.5 on 2022-08-23 00:00:00 Stop Loss: 18104.825
Exit: 17936.349609375 on 2022-09-12 00:00:00 exited on MACD condition
Profit: 503.6005859375
Entry: 17622.25 on 2022-09-19 00:00:00 Stop Loss: 18150.9175
Exit: 17314.650390625 on 2022-10-07 00:00:00 exited on MACD condition
Profit: 811.2001953125
Entry: 18244.19921875 on 2022

...

Entry: 17063.25 on 2022-02-23 00:00:00 Stop Loss: 17404.515
Exit: 16345.349609375 on 2022-03-09 00:00:00 exited on MACD condition
Profit: 717.900390625
Entry: 17784.349609375 on 2022-04-08 00:00:00 Stop Loss: 18140.0366015625
Exit: 17102.55078125 on 2022-04-29 00:00:00 exited on MACD condition
Profit: 1399.69921875
Entry: 17069.099609375 on 2022-05-02 00:00:00 Stop Loss: 17410.4816015625
Exit: 16259.2998046875 on 2022-05-17 00:00:00 exited on MACD condition
Profit: 2209.4990234375
Entry: 16478.099609375 on 2022-06-09 00:00:00 Stop Loss: 16807.6616015625
Exit: 15413.2998046875 on 2022-06-22 00:00:00 exited on MACD condition
Profit: 3274.298828125
Entry: 15938.650390625 on 2022-07-14 00:00:00 Stop Loss: 16257.4233984375
Exit: 16257.4233984375 on 2022-07-18 00:00:00 exited on stop loss
Profit: 2955.5258203124995
Entry: 17577.5 on 2022-08-23 00:00:00 Stop Loss: 17929.05
Exit: 17798.75 on 2022-09-08 00:00:00 exited on MACD condition
Profit: 2734.2758203124995
Entry: 17622.25 on 2022-09-19 0

Entry: 17063.25 on 2022-02-23 00:00:00 Stop Loss: 17404.515
Exit: 16345.349609375 on 2022-03-09 00:00:00 exited on MACD condition
Profit: 717.900390625
Entry: 17784.349609375 on 2022-04-08 00:00:00 Stop Loss: 18140.0366015625
Exit: 17102.55078125 on 2022-04-29 00:00:00 exited on MACD condition
Profit: 1399.69921875
Entry: 17069.099609375 on 2022-05-02 00:00:00 Stop Loss: 17410.4816015625
Exit: 16259.2998046875 on 2022-05-17 00:00:00 exited on MACD condition
Profit: 2209.4990234375
Entry: 16478.099609375 on 2022-06-09 00:00:00 Stop Loss: 16807.6616015625
Exit: 15413.2998046875 on 2022-06-22 00:00:00 exited on MACD condition
Profit: 3274.298828125
Entry: 15938.650390625 on 2022-07-14 00:00:00 Stop Loss: 16257.4233984375
Exit: 16257.4233984375 on 2022-07-18 00:00:00 exited on stop loss
Profit: 2955.5258203124995
Entry: 17577.5 on 2022-08-23 00:00:00 Stop Loss: 17929.05
Exit: 17798.75 on 2022-09-08 00:00:00 exited on MACD condition
Profit: 2734.2758203124995
Entry: 17622.25 on 2022-09-19 0

Entry: 17200.80078125 on 2022-04-26 00:00:00 Stop Loss: 17544.816796875
Exit: 17038.400390625 on 2022-04-27 00:00:00 exited on MACD condition
Profit: 488.4521484375
Entry: 17245.05078125 on 2022-04-28 00:00:00 Stop Loss: 17589.951796875
Exit: 17102.55078125 on 2022-04-29 00:00:00 exited on MACD condition
Profit: 630.9521484375
Entry: 17069.099609375 on 2022-05-02 00:00:00 Stop Loss: 17410.4816015625
Exit: 16259.2998046875 on 2022-05-17 00:00:00 exited on MACD condition
Profit: 1440.751953125
Entry: 16478.099609375 on 2022-06-09 00:00:00 Stop Loss: 16807.6616015625
Exit: 15413.2998046875 on 2022-06-22 00:00:00 exited on MACD condition
Profit: 2505.5517578125
Entry: 15938.650390625 on 2022-07-14 00:00:00 Stop Loss: 16257.4233984375
Exit: 16257.4233984375 on 2022-07-18 00:00:00 exited on stop loss
Profit: 2186.7787499999995
Entry: 17577.5 on 2022-08-23 00:00:00 Stop Loss: 17929.05
Exit: 17542.80078125 on 2022-09-01 00:00:00 exited on MACD condition
Profit: 2221.4779687499995
Entry: 17539.

Entry: 17063.25 on 2022-02-23 00:00:00 Stop Loss: 17404.515
Exit: 16345.349609375 on 2022-03-09 00:00:00 exited on MACD condition
Profit: 717.900390625
Entry: 17784.349609375 on 2022-04-08 00:00:00 Stop Loss: 18140.0366015625
Exit: 17102.55078125 on 2022-04-29 00:00:00 exited on MACD condition
Profit: 1399.69921875
Entry: 17069.099609375 on 2022-05-02 00:00:00 Stop Loss: 17410.4816015625
Exit: 16259.2998046875 on 2022-05-17 00:00:00 exited on MACD condition
Profit: 2209.4990234375
Entry: 16478.099609375 on 2022-06-09 00:00:00 Stop Loss: 16807.6616015625
Exit: 15413.2998046875 on 2022-06-22 00:00:00 exited on MACD condition
Profit: 3274.298828125
Entry: 15938.650390625 on 2022-07-14 00:00:00 Stop Loss: 16257.4233984375
Exit: 16257.4233984375 on 2022-07-18 00:00:00 exited on stop loss
Profit: 2955.5258203124995
Entry: 17577.5 on 2022-08-23 00:00:00 Stop Loss: 17929.05
Exit: 17798.75 on 2022-09-08 00:00:00 exited on MACD condition
Profit: 2734.2758203124995
Entry: 17622.25 on 2022-09-19 0

3 18 2 11 3
Entry: 17063.25 on 2022-02-23 00:00:00 Stop Loss: 17575.1475
Exit: 16345.349609375 on 2022-03-09 00:00:00 exited on MACD condition
Profit: 717.900390625
Entry: 17784.349609375 on 2022-04-08 00:00:00 Stop Loss: 18317.88009765625
Exit: 17102.55078125 on 2022-04-29 00:00:00 exited on MACD condition
Profit: 1399.69921875
Entry: 17069.099609375 on 2022-05-02 00:00:00 Stop Loss: 17581.17259765625
Exit: 16259.2998046875 on 2022-05-17 00:00:00 exited on MACD condition
Profit: 2209.4990234375
Entry: 16478.099609375 on 2022-06-09 00:00:00 Stop Loss: 16972.44259765625
Exit: 15413.2998046875 on 2022-06-22 00:00:00 exited on MACD condition
Profit: 3274.298828125
Entry: 15938.650390625 on 2022-07-14 00:00:00 Stop Loss: 16416.80990234375
Exit: 16278.5 on 2022-07-18 00:00:00 exited on MACD condition
Profit: 2934.44921875
Entry: 17577.5 on 2022-08-23 00:00:00 Stop Loss: 18104.825
Exit: 17798.75 on 2022-09-08 00:00:00 exited on MACD condition
Profit: 2713.19921875
Entry: 17622.25 on 2022-09-

In [26]:
test_data = yf.download("^NSEI", start='2023-01-01',end='2024-01-08')

[*********************100%%**********************]  1 of 1 completed


In [29]:
#Use the best parameters obtained from the GA
#"Best Parameters (Fast Period, Slow Period, ROC Period, RSI Period, ROC Threshold, RSI Derivative Threshold, Stop Loss %):"
#best_parameters = [2, 15, 6, 10, 0] #dlf 2year
#best_parameters = [2, 17, 2, 15, 0] # dlf 1year
#best_parameters = [4, 18, 9, 14, 0]# 1 nifty 1year
best_parameters = [3, 18, 3, 6, 6, -42, 0]# 2 nifty 1year



# Apply the strategy on out-of-sample data using the best parameters
out_of_sample_profit = evaluate_strategy(test_data.copy(), *best_parameters)


# Print or analyze the performance metrics
print("Out-of-sample profit:", out_of_sample_profit)

3 18 3 6 0
Entry: 17826.69921875 on 2023-02-21 00:00:00 Stop Loss: 17826.69921875
Exit: 17594.349609375 on 2023-03-03 00:00:00 exited on MACD condition
Profit: 232.349609375
Entry: 16985.599609375 on 2023-03-16 00:00:00 Stop Loss: 16985.599609375
Exit: 17066.599609375 on 2023-03-20 00:00:00 exited on stop loss
Profit: 151.349609375
Entry: 17618.75 on 2023-04-19 00:00:00 Stop Loss: 17618.75
Exit: 17767.30078125 on 2023-04-26 00:00:00 exited on stop loss
Profit: 2.798828125
Entry: 18314.80078125 on 2023-05-12 00:00:00 Stop Loss: 18314.80078125
Exit: 18432.349609375 on 2023-05-16 00:00:00 exited on stop loss
Profit: -114.75
Entry: 18203.400390625 on 2023-05-19 00:00:00 Stop Loss: 18203.400390625
Exit: 18294.80078125 on 2023-05-24 00:00:00 exited on stop loss
Profit: -206.150390625
Entry: 18321.150390625 on 2023-05-25 00:00:00 Stop Loss: 18321.150390625
Exit: 18368.349609375 on 2023-05-26 00:00:00 exited on stop loss
Profit: -253.349609375
Entry: 18534.099609375 on 2023-06-02 00:00:00 Stop